# Pose Estimation

Environment setup

[repo](https://github.com/ildoonet/tf-pose-estimation)

1. create conda environment
2. install opencv, tensorflow,protobuf, python3-tk
3. open project dir and `pip3 install -r requirements.txt`
4. instal swig in conda `conda install -c conda-forge/label/cf201901 swig`

Ready to run


commands to use 

test interface : `python run.py --model=mobilenet_thin --resize=432x368 --image=./images/p1.jpg`  
run on webcam  : `python run_webcam.py --model=mobilenet_thin --resize=432x368 --camera=0`  
run on video   : `python run_video.py --model=mobilenet_thin --resolution=432x368 --video=test.mp4`
 

In [ ]:
import time
import os
import cv2
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

### Test on WebCam/Video

In [ ]:
fps_time = 0

if __name__ == '__main__':
   
    description='tf-pose-estimation realtime webcam'
    camera= './demo_data/PullUps/v_PullUps_g08_c03.avi'
    resize='320x240'
    resize_out_ratio=4.0
    model_name ='mobilenet_thin'
    show_process=False
    
    model= get_graph_path(model_name)
    w, h = model_wh(resize)
    
    if w > 0 and h > 0:
        e = TfPoseEstimator(get_graph_path(model_name), target_size=(w, h))
    else:
        e = TfPoseEstimator(get_graph_path(model_name), target_size=(432, 368))
        
    cam = cv2.VideoCapture(camera)
    ret_val, image = cam.read()
    
    count = 0
    while True:
        ret_val, image = cam.read()
        count += 1
        humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
        
        blank_image = np.zeros((h,w,3), np.uint8)
        blank_image[:,:] = (255,255,255) 
        
        original = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
        blank_image = TfPoseEstimator.draw_humans(blank_image, humans, imgcopy=False)
        
        cv2.imshow('blank', blank_image)
        cv2.imshow('original', original)
        cv2.imwrite('{}_frame_%d.jpg'.format(camera) % count, image)
        fps_time = time.time()
        cv2.waitKey(100)

    cv2.destroyAllWindows()


### Test on images

In [ ]:
def image_pos_extraction(img):

    image = img
    model_name ='mobilenet_thin'
    resize_out_ratio=4.0
    show_process=False

    model= get_graph_path(model_name)
    w, h = image.shape[1], image.shape[0]

    e = TfPoseEstimator(get_graph_path(model_name), target_size=(w, h))
    
    humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)

    blank_image = np.zeros((h,w,3), np.uint8)
    blank_image[:,:] = (0,0,0) 

    image = TfPoseEstimator.draw_humans(blank_image, humans, imgcopy=False)
    cv2.imshow('img',image)
    cv2.waitKey(0)
    return image


In [ ]:
image_pos_extraction(cv2.imread('test.jpg'))

In [ ]:
data_dir = './demo_data'
image_dir= './images/'
list_dir = os.listdir(data_dir)
path = []

for directory in list_dir:
    path.append((directory,data_dir+'/{}'.format(directory)))

In [ ]:
path

In [ ]:
for category in path:
    globals()[category[0]] = os.listdir(category[1])

In [ ]:
model_name ='mobilenet_thin'
resize_out_ratio=4.0
show_process=False
model= get_graph_path(model_name)
w, h = 320,240

e = TfPoseEstimator(get_graph_path(model_name), target_size=(w, h))

for category in path:
    count = 0
    if not os.path.exists(image_dir+category[0]):
        os.makedirs(image_dir+category[0])
    for video in globals()[category[0]]:
        capture = cv2.VideoCapture(category[1]+'/{}'.format(video))
        
        fps = int(capture.get(cv2.CAP_PROP_FPS))
        size = (int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)),int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        fourcc = 0x00000021
        
        output_path = image_dir+category[0]+'/{}.mp4'.format(os.path.splitext(video)[0])
        
        writer = cv2.VideoWriter(output_path,fourcc,fps,size)
        
        success = True
        
        while success:
            try:
                count += 1
                print('Read a new frame: ', success,count)
                success,image = capture.read()

                humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)

                blank_image = np.zeros((h,w,3), np.uint8)
                blank_image[:,:] = (0,0,0)

                image = TfPoseEstimator.draw_humans(blank_image, humans, imgcopy=False)
                
                writer.write(image)
            except:
                print("Next entry.")
                writer.release()
                capture.release()
            